In [1]:
import pandas as pd
import numpy as np
import uuid
import pickle
import os
import configparser
import pymysql
import multiprocessing as mp
import sys
from imp import reload
# from pyspark.ml.fpm import FPGrowth
from Code import data_generator_new
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [2]:
reload(data_generator_new)

<module 'Code.data_generator_new' from '/home/zijun.wu/Tag_graph/Code/data_generator_new.py'>

### 数据探查

In [ ]:
data_raw = pd.read_sql("select comp_id, label_name, classify_id, label_type, label_type_num from company_tag_info_latest0703", con=data_generator_new.db)

In [ ]:
len(data_raw)

In [ ]:
comp_id_dict = data_raw.comp_id.drop_duplicates().reset_index(drop=True).reset_index() \
    .rename(index=str, columns={"index": "comp_int_id"})
data_raw = data_raw.merge(comp_id_dict, how="left", left_on="comp_id", right_on="comp_id")

In [ ]:
data_raw.head(3)

In [ ]:
ctag_table = data_raw[data_raw.classify_id != 4]

In [ ]:
ctag_agg = nctag_table.groupby("label_name").agg({"comp_id": "count"})

In [ ]:
ctag_agg.sort_values(by="comp_id", ascending=False)

In [ ]:
nctag_table = data_raw[data_raw.classify_id == 4]

In [ ]:
nctag_agg = nctag_table.groupby("label_name").agg({"comp_id": "count"}).reset_index()

In [ ]:
nctag_agg["word_len"] = nctag_agg.label_name.apply(lambda x: len(str(x)))

In [ ]:
nctag_agg.sort_values(by="comp_id", ascending=False).head(100)

In [ ]:
nctag_agg[nctag_agg.comp_id >= 150].groupby("word_len").agg({"label_name": "count"})

In [ ]:
nctag_agg[(nctag_agg.comp_id >= 150) & (nctag_agg.word_len > 3)]

In [ ]:
nctag_to_keep = nctag_agg[(nctag_agg.comp_id >= 150) & (nctag_agg.comp_id <= 1000000) & (nctag_agg.word_len >= 4)]

In [ ]:
nctag_to_keep

In [ ]:
nctag_table = nctag_table[nctag_table.label_name.isin(nctag_to_keep.label_name)]

In [ ]:
nctag_table.head(2)

In [ ]:
ctag_table.head(2)

### 检测comp_tag函数

In [ ]:
data_generator_new.comp_tag()

### 检测data_aggregator函数，返回概念、非概念各自聚合公司的结果，以及公司标签关联、公司总数

In [ ]:
data_generator_new.data_aggregator()

### 计算公司标签全集

In [ ]:
data_generator_new.properties()

### 计算标签关联

In [ ]:
%load_ext memory_profiler

In [ ]:
tag_tag_relations = data_generator_new.tag_tag2()

In [3]:
def final_count(l1, l2):
    if len(l1.union(l2)) == 0:
        return 0
    else:
        return len(l1.intersection(l2))/len(l1.union(l2))

def simple_minmax(column_target, min_v=0.001, max_v=1):
    target = column_target.values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(min_v, max_v))
    scaler.fit(target)
    return scaler.transform(target)

In [4]:
from pyspark import SQLContext, SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
sqlContext=SQLContext(sc)
spark = SparkSession.builder.getOrCreate()

In [ ]:
tag_comps_aggregated_raw = pickle.load(open("../Data/Output/Tag_graph/tag_comps_aggregated.pkl", "rb"))
tag_comps_aggregated_raw.comp_int_id = tag_comps_aggregated_raw.comp_int_id.apply(lambda x: list(x))
tag_comps_aggregated = sqlContext.createDataFrame(tag_comps_aggregated_raw)
result = tag_comps_aggregated.withColumnRenamed("comp_int_id", "comp_int_id2") \
    .withColumnRenamed("type", "type2") \
    .withColumnRenamed("tag_uuid", "tag_uuid2") \
    .crossJoin(tag_comps_aggregated) \
    .rdd.map(lambda x: (x[2], x[5]) + (x[1] + x[4], final_count(set(x[0]), set(x[3])))) \
    .distinct().toDF(["tag1", "tag2", "link_type", "link_value"]).filter("link_value > 0")
tag_tag = result.toPandas()
tag_tag.columns = ["tag1", "tag2", "link_type", "link_value"]
tag_tag.link_value = nctag_nctag.link_value.apply(lambda x: np.log2(min(0.000001 + x, 1)))
tag_tag.link_value = simple_minmax(nctag_nctag.link_value)
tag_tag = tag_tag[["tag1", "tag2", "link_type", "link_value"]].drop_duplicates()
tag_tag["tag_link"] = tag_tag.tag1 + "-" + tag_tag.tag2
tag_tag_dict = dict(zip(tag_tag.tag_link, (tag_tag.link_value, tag_tag.link_type)))

### 点和边关系储存

In [ ]:
data_generator.neo4j_merge(tag_points, company_points, comp_tag_relations, ctag_ctag_relations, ctag_nctag_relation, nctag_nctag)

### 全流程测试

In [ ]:
!python Code/pipline_new.py

### 智能网联汽车关联的top1000概念非概念标签

In [ ]:
relations = pd.read_csv("../Data/Output/Tag_graph/all_relations.csv", header=None)

In [ ]:
relations.columns = ["src_id", "target_id", "rel_value", "rel_type"]

In [ ]:
relations.head(1)

In [ ]:
tag_dict = pickle.load(open("../Data/Output/Tag_graph/tag_dict.pkl", "rb"))
tag_dict_reverse = dict(zip(tag_dict.values(), tag_dict.keys()))

In [ ]:
target_tag = tag_dict.get("智能网联汽车")

In [ ]:
target_tag

In [ ]:
linked_ctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)

In [ ]:
len(linked_ctag_top_1000)

In [ ]:
_linked_ctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)

In [ ]:
len(_linked_ctag_top_1000)

In [ ]:
linked_nctag_top_1000 = relations[(relations.target_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)[:1000]

In [ ]:
len(linked_nctag_top_1000)

In [ ]:
_linked_nctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ntag")].sort_values(by="rel_value", ascending=False)[:1000]

In [ ]:
linked_ctag_top_1000["target_tag_name"] = linked_ctag_top_1000.targe_id.apply(lambda x: tag_dict_reverse.get(x))[:1000]

In [ ]:
linked_nctag_top_1000["target_tag_name"] = linked_nctag_top_1000.targe_id.apply(lambda x: tag_dict_reverse.get(x))[:1000]

In [51]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf

In [6]:
from Code.data_generator_spark import comp_tag

SyntaxError: invalid syntax (data_generator_spark.py, line 96)

In [ ]:
reload(data_generator_spart)

In [28]:
x = sc.parallelize([["a", "x", 2, 1], ["a", "x", 3, 1], ["b", "y", 4, 1], ["b", "y", 5, 1]]).toDF(["tag", "property", "comp_id", "count"])

In [29]:
x.show()

+---+--------+-------+-----+
|tag|property|comp_id|count|
+---+--------+-------+-----+
|  a|       x|      2|    1|
|  a|       x|      3|    1|
|  b|       y|      4|    1|
|  b|       y|      5|    1|
+---+--------+-------+-----+



In [40]:
x.groupby("tag").agg(F.collect_set("comp_id").alias("comp_id"), F.max("property"), F.sum("count")).show()

+---+-------+-------------+----------+
|tag|comp_id|max(property)|sum(count)|
+---+-------+-------------+----------+
|  b| [5, 4]|            y|         2|
|  a| [2, 3]|            x|         2|
+---+-------+-------------+----------+



In [ ]:
x.groupby("tag").agg(F.collect_set("comp_id").alias("comp_id"), x.property)

In [46]:
x.join(x, "tag", "left_outer").show()

+---+--------+-------+-----+--------+-------+-----+
|tag|property|comp_id|count|property|comp_id|count|
+---+--------+-------+-----+--------+-------+-----+
|  b|       y|      4|    1|       y|      4|    1|
|  b|       y|      4|    1|       y|      5|    1|
|  b|       y|      5|    1|       y|      4|    1|
|  b|       y|      5|    1|       y|      5|    1|
|  a|       x|      2|    1|       x|      2|    1|
|  a|       x|      2|    1|       x|      3|    1|
|  a|       x|      3|    1|       x|      2|    1|
|  a|       x|      3|    1|       x|      3|    1|
+---+--------+-------+-----+--------+-------+-----+



In [47]:
y = sc.parallelize([["a", "测试", 2, 1], ["a", "猫", 3, 1], ["b", "狗狗", 4, 1], ["b", "老师", 5, 1]]).toDF(["tag", "property", "comp_id", "count"])

In [48]:
y.show()

+---+--------+-------+-----+
|tag|property|comp_id|count|
+---+--------+-------+-----+
|  a|      测试|      2|    1|
|  a|       猫|      3|    1|
|  b|      狗狗|      4|    1|
|  b|      老师|      5|    1|
+---+--------+-------+-----+



In [52]:
uuid_udf = udf(lambda x: uuid.uuid5(uuid.NAMESPACE_URL, x).hex)

In [57]:
z = y.withColumn("uuid", uuid_udf("property"))

In [55]:
y.count

<bound method DataFrame.count of DataFrame[tag: string, property: string, comp_id: bigint, count: bigint]>

In [58]:
z.dtypes

[('tag', 'string'),
 ('property', 'string'),
 ('comp_id', 'bigint'),
 ('count', 'bigint'),
 ('uuid', 'string')]

In [59]:
z.show()

+---+--------+-------+-----+--------------------+
|tag|property|comp_id|count|                uuid|
+---+--------+-------+-----+--------------------+
|  a|      测试|      2|    1|6ad47cb2231155f0a...|
|  a|       猫|      3|    1|8a93424cbff256e2a...|
|  b|      狗狗|      4|    1|a545c122b48f51f3a...|
|  b|      老师|      5|    1|cf7353129a935377a...|
+---+--------+-------+-----+--------------------+



In [61]:
def test(df):
    df = df.withColumn("uuid", uuid_udf("property"))
    return df

In [62]:
t = test(y)

In [63]:
t.show()

+---+--------+-------+-----+--------------------+
|tag|property|comp_id|count|                uuid|
+---+--------+-------+-----+--------------------+
|  a|      测试|      2|    1|6ad47cb2231155f0a...|
|  a|       猫|      3|    1|8a93424cbff256e2a...|
|  b|      狗狗|      4|    1|a545c122b48f51f3a...|
|  b|      老师|      5|    1|cf7353129a935377a...|
+---+--------+-------+-----+--------------------+



In [64]:
y.show()

+---+--------+-------+-----+
|tag|property|comp_id|count|
+---+--------+-------+-----+
|  a|      测试|      2|    1|
|  a|       猫|      3|    1|
|  b|      狗狗|      4|    1|
|  b|      老师|      5|    1|
+---+--------+-------+-----+



In [72]:
y.comp_id + 2

Column<b'(comp_id + 2)'>